In [14]:
reads = ['TGGCA', 'GCATTGCAA', 'TGCAAT', 'CAATT', 'ATTTGAC']
dna_seq = 'TGGCATTGCAATTTGAC'

# OLC 

- overlap()
- naive_overlap()
- pegar_maximo_overlap()
- olc_guloso()

In [15]:
def overlap(a, b, min_len=3):
    inicio = 0
    while True:
        inicio = a.find(b[:min_len], inicio)
        if inicio == -1:
            return 0
        elif b.startswith(a[inicio:]):
            return len(a) - inicio
        inicio += 1

print(overlap('TGGCA', 'GCATTGCAA', 3))

3


In [16]:
from itertools import permutations

def naive_overlap(reads, min_len):
    overlaps = dict()
    
    for a, b in permutations(reads, 2):
        overlap_len = overlap(a, b, min_len)
        
        if overlap_len > 0:
            overlaps[(a, b)] = overlap_len
    return overlaps

naive_overlap(reads, min_len=3)

{('TGGCA', 'GCATTGCAA'): 3,
 ('GCATTGCAA', 'TGCAAT'): 5,
 ('GCATTGCAA', 'CAATT'): 3,
 ('TGCAAT', 'CAATT'): 4,
 ('CAATT', 'ATTTGAC'): 3}

In [17]:
def pegar_maximo_overlap(reads, min_len):
    readA, readB = None, None
    melhor_overlap = 0
    
    for a,b in permutations(reads, 2):
        overlap_len = overlap(a, b, min_len)
        
        if overlap_len > melhor_overlap:
            readA, readB = a, b
            melhor_overlap = overlap_len
    
    return readA, readB, melhor_overlap

pegar_maximo_overlap(reads, min_len=3)

('GCATTGCAA', 'TGCAAT', 5)

In [18]:
def olc_guloso(reads, min_len):
    readA, readB, overlap_len = pegar_maximo_overlap(reads, min_len)
    
    while overlap_len > 0:
        reads.remove(readA)
        reads.remove(readB)
        reads.append(readA + readB[overlap_len:])
        
        readA, readB,overlap_len = pegar_maximo_overlap(reads, min_len)
    
    return ''.join(reads)

olc_guloso(reads, min_len=3)

'TGGCATTGCAATTTGAC'

In [19]:
# limitação - rigidez

# Grafos de Brujin 

A ideia principal do algoritmo é trabalhar com a sobreposição de k-mers. Vamos trabalhar somente com as funções de_brujin_graph() e visualizar_de_brujin().

In [21]:
print(dna_seq)
# Desafio: trabalhar a partir da sequencia de dna pra pegar o grafo pra entender a logica

TGGCATTGCAATTTGAC


In [24]:
def de_brujin_graph(dna_seq, kmer):
    vertices = set()
    arestas = list()
    
    for i in range(len(dna_seq) - kmer + 1):
        vertices.add(dna_seq[i: i+kmer-1])
        vertices.add(dna_seq[i+1: i+kmer])
        
        arestas.append((dna_seq[i:i+kmer-1], dna_seq[i+1: i+kmer]))
    
    return vertices, arestas

In [26]:
vertices, arestas = de_brujin_graph(dna_seq, 4)

In [27]:
print(f'Os vértices são {vertices}')

Os vértices são {'AAT', 'TGC', 'GCA', 'TGG', 'TTG', 'GGC', 'TTT', 'GAC', 'ATT', 'CAT', 'CAA', 'TGA'}


In [28]:
print(f'As arestas são {arestas}')

As arestas são [('TGG', 'GGC'), ('GGC', 'GCA'), ('GCA', 'CAT'), ('CAT', 'ATT'), ('ATT', 'TTG'), ('TTG', 'TGC'), ('TGC', 'GCA'), ('GCA', 'CAA'), ('CAA', 'AAT'), ('AAT', 'ATT'), ('ATT', 'TTT'), ('TTT', 'TTG'), ('TTG', 'TGA'), ('TGA', 'GAC')]


In [33]:
def visualizar_de_brujin(dna_seq, kmer):
    vertices, arestas = de_brujin_graph(dna_seq, kmer)
    
    dot_str = "digraph \"DeBrujin graph\" {\n"
    
    for v in vertices:
        dot_str += f' {v} [label="{v}"];\n'
                                          
    for fonte, destino in arestas:
        dot_str += f' {fonte} -> {destino};\n'
    
    dot_str += '}\n'
    return dot_str

In [35]:
print(visualizar_de_brujin(dna_seq, 4))

digraph "DeBrujin graph" {
 AAT [label="AAT"];
 TGC [label="TGC"];
 GCA [label="GCA"];
 TGG [label="TGG"];
 TTG [label="TTG"];
 GGC [label="GGC"];
 TTT [label="TTT"];
 GAC [label="GAC"];
 ATT [label="ATT"];
 CAT [label="CAT"];
 CAA [label="CAA"];
 TGA [label="TGA"];
 TGG -> GGC;
 GGC -> GCA;
 GCA -> CAT;
 CAT -> ATT;
 ATT -> TTG;
 TTG -> TGC;
 TGC -> GCA;
 GCA -> CAA;
 CAA -> AAT;
 AAT -> ATT;
 ATT -> TTT;
 TTT -> TTG;
 TTG -> TGA;
 TGA -> GAC;
}



In [36]:
%dotstr visualizar_de_brujin(dna_seq, 4)

UsageError: Line magic function `%dotstr` not found.


**limitação**: o grafo foi feito a partir da sequencia (dna_seq). O que se espera é utilizar cada kmer e depois implementar um algoritmo de caminho Euleriano.